In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
import tensorflow as tf
from tqdm import tqdm
import utils

# Import data 
game_state_20, game_state_40, game_state_60, game_state_80, game_state_100 = utils.get_inputs()

2023-11-22 21:15:39.346967: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 21:15:39.385167: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 21:15:39.385197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 21:15:39.386561: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 21:15:39.392977: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 21:15:39.393305: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# Prepare training, validation, and test datasets for 20% game time

train_frac = 0.8
valid_frac = 0.1

train_set = game_state_20.sample(frac = 0.8)
remaining_set = game_state_20.drop(train_set.index)
valid_set = remaining_set.sample(frac = valid_frac / (1 - train_frac))
test_set = remaining_set.drop(valid_set.index)

# Separate wins
train_wins = train_set.pop('blueWin')
valid_wins = valid_set.pop('blueWin')
test_wins = test_set.pop('blueWin')

print(game_state_20.shape, train_set.shape, valid_set.shape, test_set.shape)


(64556, 36) (51645, 35) (6456, 35) (6455, 35)


In [4]:
### Try simple DNN classification

# Convert dataframes to tensors
train_data = tf.convert_to_tensor(train_set)
train_labels = tf.convert_to_tensor(train_wins)

valid_data = tf.convert_to_tensor(valid_set)
valid_labels = tf.convert_to_tensor(valid_wins)

#train_data = train_data[..., tf.newaxis]
#valid_data = valid_data[..., tf.newaxis]
# Define batches

def get_batch(data, labels, batch_size):
    return tf.data.Dataset.from_tensor_slices((data, labels)).batch(batch_size)

train_batched = get_batch(train_data, train_labels, 64)
valid_batched = get_batch(valid_data, valid_labels, 64)

print(tf.shape(train_data), tf.shape(train_labels))


tf.Tensor([51645    35], shape=(2,), dtype=int32) tf.Tensor([51645], shape=(1,), dtype=int32)


In [ ]:
# Define model
class DNNModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        # Single dense layer
        self.layer1 = tf.keras.layers.Dense(35, activation='relu')
        self.layer2 = tf.keras.layers.Dense(1)
        
    def call(self, input):
        input = self.layer1(input)
        return self.layer2(input)

model = DNNModel()

# Choose optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_func = tf.keras.losses.CategoricalCrossentropy()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='validation_loss')
valid_accuracy = tf.keras.metrics.CategoricalAccuracy(name='validation_accuracy')


# Define training step
@tf.function
def train_step(match_states, outcomes):
    with tf.GradientTape() as tape:  
        predictions = tf.squeeze(model(match_states))
        loss = loss_func(outcomes, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(outcomes, predictions)

# Define testing step
@tf.function
def test_step(match_states, outcomes):
    predictions = tf.squeeze(model(match_states))
    loss = loss_func(outcomes, predictions)

    valid_loss(loss)
    valid_accuracy(outcomes, predictions)

# Set output
output_dir = './output'
output_prefix = os.path.join(output_dir, "DNN_2l_20")


# Run
EPOCHS = 100

for epoch in tqdm(range(EPOCHS)):
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

    for match_states, outcomes in train_batched:
        train_step(match_states, outcomes)

    for match_states, outcomes in valid_batched:
        test_step(match_states, outcomes)

    if epoch % 10 == 0:
        model.save_weights(output_prefix)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result()}, '
    f'Validation Loss: {valid_loss.result()}, '
    f'Validation Accuracy: {valid_accuracy.result()}'
  )

model.save_weights(output_prefix)

  1%|▍                                          | 1/100 [00:02<03:38,  2.21s/it]

Epoch 1, Loss: 135.4866485595703, Accuracy: 0.012391573749482632, Validation Loss: 133.3809051513672, Validation Accuracy: 0.0


  2%|▊                                          | 2/100 [00:02<02:13,  1.37s/it]

Epoch 2, Loss: 135.02342224121094, Accuracy: 0.01610904559493065, Validation Loss: 133.60118103027344, Validation Accuracy: 0.0


  3%|█▎                                         | 3/100 [00:03<01:46,  1.10s/it]

...

Epoch 35, Loss: 133.46229553222656, Accuracy: 0.014869888313114643, Validation Loss: 131.6865692138672, Validation Accuracy: 0.009900989942252636


 36%|███████████████                           | 36/100 [00:29<00:51,  1.24it/s]

Epoch 36, Loss: 133.45742797851562, Accuracy: 0.013630731031298637, Validation Loss: 131.693603515625, Validation Accuracy: 0.009900989942252636


 37%|███████████████▌                          | 37/100 [00:31<01:00,  1.03it/s]

Epoch 37, Loss: 133.46646118164062, Accuracy: 0.014869888313114643, Validation Loss: 131.67564392089844, Validation Accuracy: 0.009900989942252636
